                                         Case Study: Problem-1 [ 3-4 Hours]
A shop-keeper would like to decide weekly pricing of their products. He/She generally plans pricing for each quarter and would like to achieve following:
1. Clearout at least 60% of total inventory.
2. achieve total margin across all products (i.e. selling price - cost price).
Following are a few things to consider while deciding pricing of products.
3. There shouldn’t be much difference in the pricing from one week to another.
4. A particular price selected applies for all the products in the group.
                                                    Expectations
1. Develop a mathematical model that can help shop-keeper to identify their pricing for a quarter (13 weeks in the provided example)
2. Model has to suggest a pricing (price-point) that will be applied across all the products to achieve highest possible margin considering inventory clearance of 60% (total inventory in the provided example is 264443.0 units, 60% of this has to be sold)
3. Max Price change from one week to next is 20 units
4. Shop-keeper has to cater posed weekly demand to customers if inventory is available
5. A single price-point for a week applies to all the products as they belong to same group
6. Implementation of model in python and obtain some initial results
7. Algorithmic details to solve the model and provide insights on complexity of model in scaling to larger number of products/weeks.

# Data Cleaning and Summary

In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict

In [2]:
data=pd.read_csv('/Users/Z003CNF/Desktop/AssignmentData.csv')

In [27]:
## check size of data and nulls if any 
data.shape

(1265745, 8)

In [10]:
data['product_id'].nunique()

6491

In [28]:
1265745/6491 # one product has 195 rows 

195.0

In [23]:
d1['price'].nunique()

15

In [29]:
13*15    # for each prod in each week 15 differnet price points
          #are there of which we need to select one price point to maximize total margin 
          #and demand is atleast 60% of inv

195

In [34]:
a=data['product_id'].value_counts() # to confirm if total 195 rows for each prod.
a.nunique()

1

In [36]:
# Handling missing values if any

# Exploratory Data Analysis

In [5]:
# Total Inventory for all prods at the start of quater

prod_inv=data.groupby(['product_id']).agg(inventory=pd.NamedAgg(column="total_inventory", aggfunc="mean")).reset_index()
prod_inv['inventory'].sum()


264443.0

In [44]:
0.6*264443 # This much inventory needs to be cleared

158665.8

In [65]:
# check if sp is always > cp for all prod's and possible price points
data['is_sp_gt_cp']=np.where(data['selling_price'] >= data['cost_price'],1,0)
data['is_sp_gt_cp'].sum()

1265745

In [ ]:
## Selling price was always > cost price hence we will always have positive margin.

In [3]:
# check if total quater demand is less than inv. available at start of quater
# taken max of a demand over all options in a week
prod_week_demand=data.groupby(['product_id','week']).agg(Demand=pd.NamedAgg(column="demand", aggfunc="max"))
prod_demand=prod_week_demand.groupby(['product_id']).agg(Tot_Demand=pd.NamedAgg(column="Demand", aggfunc="sum"))
prod_demand.sum()

Tot_Demand    620285.0
dtype: float64

In [6]:
# Compare inventory and demand
# maximum demand scenerio is 0.6 M but overall inventory is 0.26 M , so if all the products have high demand scenerio
# we may not be able to fulfill all of the demand.

prods=pd.merge(prod_inv,prod_demand, on=["product_id"])
prods['if']=np.where(prods['inventory'] >= prods['Tot_Demand'],1,0)
prods['if'].sum()

1697

In [7]:
prods

,product_id,inventory,Tot_Demand,if
0,1,50.0,7.0,1
1,196,31.0,16.0,1
2,391,32.0,19.0,1
3,586,15.0,5.0,1
4,781,16.0,5.0,1
...,...,...,...,...
6486,1264771,37.0,433.0,0
6487,1264966,61.0,703.0,0
6488,1265161,56.0,503.0,0
6489,1265356,61.0,0.0,1


In [9]:
df=prods.loc[prods['if']==1]
df

,product_id,inventory,Tot_Demand,if
0,1,50.0,7.0,1
1,196,31.0,16.0,1
2,391,32.0,19.0,1
3,586,15.0,5.0,1
4,781,16.0,5.0,1
...,...,...,...,...
6433,1254436,1.0,0.0,1
6444,1256581,2.0,1.0,1
6479,1263406,1.0,0.0,1
6489,1265356,61.0,0.0,1


In [62]:
#of 6k products, we have enough invnetory for 1.6 k products, if we consider maximum demand scenerios

### Objective Function

#####  $ Maximize: demand*(sp*x[i,j,k]-cp*x[i,j,k]) $

### Requirement 1: Inventory Clearence
##### Demand will vary as per the price we set and it will be met/un-met based on available inventory
##### $ total demand >= 158665.8 $
##### and total demand at particular price point if selected (for whole quater) <= inv. available start of quater

### Requirement 2: WOW price variation
##### $ For all i : abs(X[i,j,k]-X[i,j+1,k]) < 20    # it will be a non-linear constraint so will linearize it by added two constraints <20 & >20 $

# Optimization Model 

###### Find the optimum prices for products based on above two requirements

In [10]:
data['margin']=data['selling_price']-data['cost_price']

In [11]:
# test data for 2 products
#products=data['product_id'].unique()
products=df['product_id'].unique()
weeks=[202136, 202142, 202137, 202139, 202146, 202138, 202140, 202141,202147, 202143, 202144, 202145, 202148]
prices=[ 5.0, 10.0, 55.0, 60.0, 65.0, 70.0, 75.0, 15.0, 20.0, 25.0, 30.0, 35.0, 40.0, 45.0, 50.0]

combos = [(p, w, a) for p in products for w in weeks for a in prices]

In [12]:
data1=data.loc[data['product_id'].isin(products)]
dict_demand=data1.set_index(['product_id','week','price']).demand.to_dict()
dict_margin=data1.set_index(['product_id','week','price']).margin.to_dict()
prod_inv1=prod_inv.loc[prod_inv['product_id'].isin(products)]
dict_inv_quater=prod_inv1.set_index(['product_id']).inventory.to_dict()

In [13]:
from pulp import *
prob = LpProblem("Pricing_Problem",LpMaximize)
dvar = LpVariable.dicts("price_points",combos, cat="Binary")


In [16]:
## objective function:

prob += (
    lpSum([dvar[i] * dict_margin[i] * dict_demand[i] for i in combos])
)


In [17]:
## Constraint 1 --- Only. one price point is to be chosen for a product in a week

for p in products:
    for w in weeks:
        prob += (
            lpSum([dvar[(p,w,a)] for a in prices]) == 1
        )


In [18]:
## Constraint 2 ---- Demand should be as much as to clear 60% inventory

prob += (
    lpSum([dvar[i] * dict_demand[i] for i in combos]) >= 158665
)


In [19]:
## Constraint 3 ---- The price change from a week to next < 20 

for p in products:
    prob += (
        lpSum([dvar[(p, w, j)] for w in weeks for j in prices]) <= lpSum([dvar[(p, w+1, j)] for w in weeks for j in prices if w <= 202147]) + 20
    )

for p in products:
    prob += (
        lpSum([dvar[(p, w+1, j)] for w in weeks for j in prices if w <= 202147]) <=  lpSum([dvar[(p, w, j)] for w in weeks for j in prices]) + 20
    )
    

In [20]:
## Constraint 4 ---- Demand to be met if enough inventory is available at start of quater.

for p in products:
    prob += (
        lpSum([dvar[(p,w,a)] for w in weeks for a in prices]) <= dict_inv_quater[p]
    )

In [21]:
prob.writeLP("pricing.lp")

prob.solve()

print("Status:", LpStatus[prob.status])



Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/Z003CNF/opt/anaconda3/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/my/1126m1255k36stt7f2r98h_w0000gq/T/a1ea676b2bde44aa81b700483e107b04-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/my/1126m1255k36stt7f2r98h_w0000gq/T/a1ea676b2bde44aa81b700483e107b04-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 27158 COLUMNS
At line 2193875 RHS
At line 2221029 BOUNDS
At line 2551945 ENDATA
Problem MODEL has 27153 rows, 330915 columns and 803353 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.17 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       1.25   (Wallclock seconds):       1.41

Status: Infeasible


In [36]:
### Not used in model-----------------## Constraint 4---- Demand to be met if enough inventory is available at start of quater.

for p in products:
    prob += (
        lpSum([dvar[(p,w,a)]*dict_demand[(p,w,a)] for w in weeks for a in prices]) <= dict_inv_quater[p]
    )
    

In [72]:
output = []
for v in prob.variables():
    if v.varValue > 0.01 :
        output.append(str(v.name) + "=" + str(v.varValue))
        
df = pd.DataFrame({'col': output})

In [70]:
output

['price_points_(1,_202136,_15.0)=1.0',
 'price_points_(1,_202137,_15.0)=1.0',
 'price_points_(1,_202138,_20.0)=1.0',
 'price_points_(1,_202139,_25.0)=1.0',
 'price_points_(1,_202140,_60.0)=1.0',
 'price_points_(1,_202141,_45.0)=1.0',
 'price_points_(1,_202142,_35.0)=1.0',
 'price_points_(1,_202143,_45.0)=1.0',
 'price_points_(1,_202144,_5.0)=1.0',
 'price_points_(1,_202145,_10.0)=1.0',
 'price_points_(1,_202146,_40.0)=1.0',
 'price_points_(1,_202147,_60.0)=1.0',
 'price_points_(1,_202148,_75.0)=1.0',
 'price_points_(1000156,_202136,_40.0)=1.0',
 'price_points_(1000156,_202137,_5.0)=1.0',
 'price_points_(1000156,_202138,_10.0)=1.0',
 'price_points_(1000156,_202139,_30.0)=1.0',
 'price_points_(1000156,_202140,_35.0)=1.0',
 'price_points_(1000156,_202141,_35.0)=1.0',
 'price_points_(1000156,_202142,_35.0)=1.0',
 'price_points_(1000156,_202143,_35.0)=1.0',
 'price_points_(1000156,_202144,_35.0)=1.0',
 'price_points_(1000156,_202145,_35.0)=1.0',
 'price_points_(1000156,_202146,_35.0)=1.0',


In [73]:
df.to_excel('Result_prices.xlsx')